## importing libraries

In [50]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten
import os
import numpy as np

## making dictionary

In [ ]:
df=pd.read_csv(r"C:\Users\srish\Downloads\driver_imgs_list.csv")
data={}
## here we are iterating over the csv file rowwise and club all the image files that belong to a particular class
for index,row in df.iterrows():
    if row['classname'] not in data:
        data[row['classname']]=[row['img']]
    else:
        data[row['classname']].append(row['img'])

In [6]:
## we do this to find out the no of categories
emotion_list=list(data.keys())
print(emotion_list)

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']


## creating model

In [19]:
## we create a model with a convulation layer of 16 units then another of 32 units another 64 units and a hidden layer of 512 units
## output layers has 10 units as we have have 10 categories 
models=tf.keras.Sequential([
    Conv2D(16,(3,3),activation="relu",input_shape=(100,100,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512,activation="relu"),
    Dense(10,activation="softmax")
])
models.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['acc'])
print(models.summary())

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)                   │ (None, 98, 98, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 49, 49, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 47, 47, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_20 (MaxPooling2D)      │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 512)                 │       3,277,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,306,026 (12.61 MB)

 Trainable params: 3,306,026 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

None


##  loading the train and test directoris

In [20]:
train_dir=r"C:\Users\srish\Downloads\imgs\train"
test_dir=r"C:\Users\srish\Downloads\imgs\test"

## training the model using image data generator

In [39]:
## here our train dir  was already divided into sub directories
## so we split it as out of 10 8 will be used for training and 2 for validation so validation is 0.2
## as we use the same directories so we have added subsets 
## as we have 10 categories so class mode is categorical
train_imgdagen=ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator=train_imgdagen.flow_from_directory(
                                                    train_dir,
                                                    target_size=[100,100],
                                                    class_mode='categorical',
                                                    batch_size=128,
                                                    subset='training'
                                                      )

validation_generator=train_imgdagen.flow_from_directory(
                                                    train_dir,
                                                    target_size=[100,100],
                                                    class_mode='categorical',
                                                    batch_size=128,
                                                    subset='training'
                                                      )

Found 17943 images belonging to 10 classes.
Found 17943 images belonging to 10 classes.


In [37]:
## it is used to stop the model running if accuracy score is not improving further
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

## model fitting

In [40]:
history = models.fit(
    train_generator,
    verbose=1,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[es]
)

Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 181s 1s/step - acc: 0.9705 - loss: 0.0977 - val_acc: 0.9868 - val_loss: 0.0429
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 182s 1s/step - acc: 0.9824 - loss: 0.0565 - val_acc: 0.9698 - val_loss: 0.0878
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 174s 1s/step - acc: 0.9823 - loss: 0.0544 - val_acc: 0.9851 - val_loss: 0.0454


## testing data

In [47]:
image_files = os.listdir(test_dir)
image_paths = [os.path.join(test_dir, img_file) for img_file in image_files]

    # Create a DataFrame with filenames
test_df = pd.DataFrame({'filename': image_paths})

# Use ImageDataGenerator for rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of data from the test directory without labels (class_mode=None)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode=None,  # No labels are needed for prediction
    shuffle=False  # Do not shuffle for prediction
)

# Predict classes for test data
predictions = models.predict(test_generator, steps=len(test_generator), verbose=1)


Found 79726 validated image filenames.
623/623 ━━━━━━━━━━━━━━━━━━━━ 1283s 2s/step


## printing predictions

In [1]:
print(predictions)

NameError: name 'predictions' is not defined